In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [16]:
trajectories = pd.read_csv('./data/processed_trajectories.csv')
zn_pos = pd.read_csv('./data/zones_centroids.csv')

In [17]:
zone_coords = zn_pos.set_index('zone')[['latitude', 'longitude']].to_dict(orient='index')

# Función para obtener latitud y longitud de una zona
def get_lat_long(zone):
    if zone in zone_coords:
        return zone_coords[zone]['latitude'], zone_coords[zone]['longitude']
    else:
        return None, None

# Crear las identificaciones únicas de las torres
towers_id = np.arange(1, 135, 1)  # Asegúrate de que esto coincida con las zonas válidas en tu conjunto de datos
n_towers = len(towers_id)
codes_onehot = np.eye(n_towers)  # Generar codificación one-hot para las torres

# Crear una lista de coordenadas para todas las zonas
ll = {zone: get_lat_long(zone) for zone in towers_id}
ll[None] = (None, None)  # Añadir coordenadas nulas para zonas no válidas

# Función para generar entradas y etiquetas
def input_labels(data):
    """
    Generar entradas (`inputs`) y etiquetas (`predict`) a partir de los datos procesados.
    """
    inputs = []
    predict = []

    for _, row in data.iterrows():
        zone_start, zone_middle, zone_end, f_time = row['zone_start'], row['zone_middle'], row['zone_end'], row['f_time']
        
        # Obtener coordenadas
        lat_start, long_start = ll.get(zone_start, (None, None))
        lat_end, long_end = ll.get(zone_end, (None, None))
        
        # Verificar si las coordenadas son válidas
        if None in (lat_start, long_start, lat_end, long_end):
            continue  # Ignorar trayectorias con zonas inválidas
        
        zone_middle = int(zone_middle)  # Convertir a entero
        
        # Validar que el índice sea válido dentro de codes_onehot
        if 1 <= zone_middle <= len(codes_onehot):
            # Crear entrada
            inputs.append([long_start, lat_start, long_end, lat_end, f_time])
            
            # Crear etiqueta one-hot para la zona intermedia
            predict.append(codes_onehot[zone_middle - 1])
    
    return np.array(inputs), np.array(predict)

# Generar datos de entrada y etiquetas
inputs_v2, predict = input_labels(trajectories)

# Barajar los datos para aleatoriedad
indices = np.arange(len(inputs_v2))
np.random.shuffle(indices)
inputs_v2 = inputs_v2[indices]
predict = predict[indices]

In [18]:
scaler = MinMaxScaler()
scaler.fit(inputs_v2[:, :4])

inputs = np.zeros((inputs_v2.shape[0], 5))
inputs[:, :4] = scaler.transform(inputs_v2[:, :4])
inputs[:, 4] = inputs_v2[:, 4]

In [19]:
X_train, X_temp, y_train, y_temp = train_test_split(inputs, predict, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [20]:
# Guardar X_train y y_train
pd.DataFrame(X_train).to_csv("./data/splited/X_train.csv", index=False, header=False)
pd.DataFrame(y_train).to_csv("./data/splited/y_train.csv", index=False, header=False)

# Guardar X_valid y y_valid
pd.DataFrame(X_valid).to_csv("./data/splited/X_valid.csv", index=False, header=False)
pd.DataFrame(y_valid).to_csv("./data/splited/y_valid.csv", index=False, header=False)

# Guardar X_test y y_test
pd.DataFrame(X_test).to_csv("./data/splited/X_test.csv", index=False, header=False)
pd.DataFrame(y_test).to_csv("./data/splited/y_test.csv", index=False, header=False)